<a href="https://colab.research.google.com/github/jeyasri-s2/SJSU297-AdvancedDL/blob/master/Assignment4/Assignment4_MultiTaskLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal:

* Using AutoKeras to implement **Multi-task learning** using ***MNIST dataset*** for Image classification and Image Regression

In [ ]:
!pip install autokeras
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc4


     |████████████████████████████████| 143kB 4.2MB/s 
     |████████████████████████████████| 849kB 6.7MB/s 
     |████████████████████████████████| 358kB 8.6MB/s 
     |████████████████████████████████| 36.7MB 127kB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 1.1MB 23.7MB/s 
     |████████████████████████████████| 174kB 43.9MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=269d4b605ec257d9b8b82bff21495ec54c7741a8821c381625016408db8b4e27
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo
  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc4) to /tmp/pip-req-build-m2u8s7wj
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-m2u8s7wj
  Running command git checkout -q 3189ab613e022d53c511348a10aed69d821f3cd1
  Created wheel for keras

# Load data

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:100]
y_train = y_train[:100]
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(100, 28, 28)
(100,)
[5 0 4]


## MNIST Dataset - Image regression

In [ ]:
regression_input_node = ak.ImageInput()
regression_output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=False,
    # Do not do data augmentation.
    augment=False,
)(regression_input_node)
regression_output_node = ak.RegressionHead(metrics=['mae'])(regression_output_node)


In [ ]:
input_node = ak.ImageInput()

output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.RegressionHead(metrics=['mae'])(output_node)


## MNIST Dataset - Image classification

In [ ]:
classify_input_node = ak.ImageInput()
classify_output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(classify_input_node)
classify_output_node = ak.ClassificationHead(loss='categorical_crossentropy', metrics=['accuracy'])(classify_output_node)


# Using Automodel to train two tasks parallely

In [ ]:
model = ak.AutoModel(
    inputs=[input_node, classify_input_node],
    outputs=[
        output_node,#ak.RegressionHead(metrics=['mae']),
        classify_output_node#ak.ClassificationHead(loss='categorical_crossentropy', metrics=['accuracy'])
    ],
    overwrite=True,
    max_trials=2)

# Train model

In [ ]:
history = model.fit([x_train,x_train], [y_train,y_train], epochs=20)
history

Trial 2 Complete [00h 04m 33s]
val_loss: 5.986042022705078

Best val_loss So Far: 5.986042022705078
Total elapsed time: 00h 09m 19s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
4/4 [==============================] - 8s 2s/step - loss: 26.2999 - regression_head_1_loss: 22.3519 - classification_head_1_loss: 3.9481 - regression_head_1_mae: 3.8706 - classification_head_1_accuracy: 0.1100
Epoch 2/20
4/4 [==============================] - 8s 2s/step - loss: 12.6793 - regression_head_1_loss: 8.8603 - classification_head_1_loss: 3.8190 - regression_head_1_mae: 2.3358 - classification_head_1_accuracy: 0.2300
Epoch 3/20
4/4 [==============================] - 8s 2s/step - loss: 15.0958 - regression_head_1_loss: 11.3919 - classification_head_1_loss: 3.7039 - regression_head_1_mae: 2.7794 - classification_head_1_accuracy: 0.3100
Epoch 4/20
4/4 [==============================] - 8s 2s/step - loss: 12.9189 - regression_head_1_loss: 10.9706 - classification_head_1_loss: 1.9482 - regression_head_1_

# Conclusion

I have tried 3 trials in Auto keras and the task is executed parallely for Image classification and Image Regression

4/4 [==============================] - 8s 2s/step - loss: 1.5165 - regression_head_1_loss: 1.5162 - classification_head_1_loss: 3.0108e-04 - regression_head_1_mae: 0.9649 - classification_head_1_accuracy: 1.0000
